In [132]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import psycopg2

Is there a statistical difference between the ratings of two different music genres? 
Is there a difference between the ratings of {insert genre here} music and all other music?
Are the albums from one label rated differently than the wider population?

In [231]:
#create connection
conn = psycopg2.connect("dbname=pitchfork_reviews")

In [232]:
cur = conn.cursor()

In [233]:
#execute simple commands
cur.execute("""
SELECT * FROM pg_catalog.pg_tables;
""")

In [234]:
i = cur.fetchall()
#load cast genres into a dataframe
df = pd.DataFrame(i)

In [235]:
df.columns = [i[0] for i in cur.description]
df.head(5)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,reviews,lee,None,False,False,False,False
1,public,artists,lee,None,False,False,False,False
2,public,genres,lee,None,False,False,False,False
3,public,labels,lee,None,False,False,False,False
4,public,years,lee,None,False,False,False,False


In [236]:
#execute simple commands
cur.execute("""
SELECT * FROM genres;
""")

In [237]:
i = cur.fetchall()
#load cast genres into a dataframe
df = pd.DataFrame(i)

In [238]:
#check unique values in genres
df[1].unique()

array(['electronic', 'metal', 'rock', None, 'rap', 'experimental',
       'pop/r&b', 'folk/country', 'jazz', 'global'], dtype=object)

In [ ]:
df.columns = [i[0] for i in cur.description]
df

In [ ]:
df_genres.head(10)

In [ ]:
df_genres.info()

In [239]:
#execute simple commands
reviews = cur.execute("""
SELECT * FROM reviews;
""")

# SELECT c1, aggregate(c2)
# FROM t
# GROUP BY c1;

In [ ]:
reviews = cur.fetchall()
#load cast genres into a dataframe
df_reviews = pd.DataFrame(reviews)

In [240]:
df_reviews.columns = [i[0] for i in cur.description]

In [241]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 13 columns):
reviewid          18393 non-null int64
title             18393 non-null object
artist            18393 non-null object
url               18393 non-null object
score             18393 non-null float64
best_new_music    18393 non-null int64
author            18393 non-null object
author_type       14487 non-null object
pub_date          18393 non-null object
pub_weekday       18393 non-null int64
pub_day           18393 non-null int64
pub_month         18393 non-null int64
pub_year          18393 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.8+ MB


In [245]:
df_reviews['author_type'].value_counts()

contributor                  12422
senior editor                486  
executive editor             475  
senior staff writer          439  
contributing editor          210  
associate staff writer       105  
deputy news editor           100  
editor-in-chief              85   
tracks coordinator           52   
associate reviews editor     41   
associate editor             37   
assistant editor             16   
associate features editor    11   
news director                5    
managing editor              3    
Name: author_type, dtype: int64

In [5]:
#execute simple commands
query = cur.execute("""
SELECT AVG(score), author FROM reviews GROUP BY author;
""")

In [6]:
fetch = cur.fetchall()
#load cast genres into a dataframe
df_query = pd.DataFrame(fetch)

In [8]:
df_query.columns = [i[0] for i in cur.description]

In [9]:
df_query.head(10)

,avg,author
0,6.150000,jesse fahnestock
1,7.900000,edwin stats houghton
2,8.000000,the pitchfork staff
3,7.283871,kim kelly
4,6.500000,stephen may
5,7.385075,andy o'connor
6,6.138889,michael idov
7,6.580882,eric harvey
8,7.680000,kris ex
9,7.400000,minna zhou


In [10]:
#execute simple commands
query = cur.execute("""
SELECT AVG(score), COUNT(score) FROM reviews GROUP BY author;
""")

In [11]:
fetch = cur.fetchall()
#load cast genres into a dataframe
df_query_1 = pd.DataFrame(fetch)

In [12]:
df_query_1.columns = [i[0] for i in cur.description]

In [13]:
df_query_1.head(10)

,avg,count
0,6.150000,2
1,7.900000,1
2,8.000000,1
3,7.283871,62
4,6.500000,1
5,7.385075,67
6,6.138889,36
7,6.580882,136
8,7.680000,15
9,7.400000,1


In [ ]:
#### drop duplicates

# df.drop_duplicates(subset = 'id', keep='first', inplace=True)

In [154]:
#execute simple commands
genres = cur.execute("""
SELECT reviews.reviewid, reviews.score, reviews.author, genres.genre FROM reviews
INNER JOIN genres ON reviews.reviewid = genres.reviewid
WHERE genres.genre = 'metal' OR genres.genre = 'jazz' GROUP BY reviews.reviewid, reviews.score, reviews.author, genres.genre;
""")

In [155]:
reviews_genres_inner = cur.fetchall()
#load cast genres into a dataframe
df_genre_reviews = pd.DataFrame(reviews_genres_inner)

In [156]:
df_genre_reviews.columns = [i[0] for i in cur.description]

In [157]:
#### drop duplicates

# df_genre_reviews.drop_duplicates(subset = 'id', keep='first', inplace=True)

In [158]:
df_genre_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 4 columns):
reviewid    1294 non-null int64
score       1294 non-null float64
author      1294 non-null object
genre       1294 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 40.5+ KB


In [159]:
df_genre_reviews.head()

,reviewid,score,author,genre
0,1509,7.9,brent s. sirota,jazz
1,9421,6.0,mark richardson,jazz
2,3833,7.6,mark richardson,jazz
3,7120,7.2,matthew murphy,jazz
4,6885,4.0,jason crock,metal


In [112]:
df_genre_reviews.groupby(['author', 'genre']).mean()

# data.groupby(func, axis=0).mean()
# data.groupby(['col1', 'col2'])['col3'].mean()

reviewid     score
author                        genre                        
aaron leitko                  jazz   16213.600000  7.600000
                              metal  14036.666667  6.200000
adam dlugacz                  metal   6794.000000  6.300000
adam moerder                  jazz    5427.000000  2.700000
                              metal   6318.166667  5.866667
al shipley                    jazz    7003.000000  5.450000
alex linhardt                 jazz    2017.500000  8.150000
                              metal   8383.000000  9.700000
alexander lloyd linhardt      jazz    4770.000000  8.066667
                              metal  11691.000000  9.000000
alison fields                 jazz    5395.000000  6.500000
                              metal   4775.000000  5.860000
amanda petrusich              metal   4878.666667  5.900000
amanda petrusich              jazz   21797.000000  9.000000
amy granzin                   metal  14469.000000  7.700000
andrew bryant                 jazz    3348.500000  6.750000
                              metal   3879.714286  6.857143
andrew gaerig                 jazz   13740.666667  5.966667
                              metal  11931.000000  6.750000
andrew goldman                metal    691.000000  7.800000
andy battaglia                jazz   12361.500000  6.700000
andy beta                     jazz    8147.200000  7.780000
andy o' connor                metal  20788.000000  7.300000
andy o'connor                 jazz   18459.000000  7.700000
                              metal  20663.969697  7.363636
austin gaines                 metal   6588.750000  6.325000
ben westhoff                  jazz   11114.000000  7.500000
brad haywood                  jazz    3413.500000  4.000000
                              metal   3823.875000  7.287500
brad haywood & ryan schreiber metal   3433.000000  3.100000
...                                           ...       ...
samir khan                    jazz    7119.000000  8.000000
savy reyes-kulkarni           metal  22559.000000  7.700000
scott hreha                   jazz    4907.500000  6.150000
                              metal   8099.000000  8.400000
scott plagenhoef              jazz    8045.000000  5.950000
                              metal   8616.000000  8.200000
sean fennessey                jazz    8114.333333  7.466667
                              metal  14515.000000  8.600000
seth colter walls             jazz   21807.136364  7.931818
seth colter-walls             jazz   22707.000000  9.000000
spencer owen                  jazz    8555.000000  8.000000
                              metal   2860.000000  8.500000
stephen m. deusner            jazz   13568.000000  7.475000
                              metal  13522.000000  6.450000
stephen trouss                jazz   11780.000000  7.400000
steven hyden                  metal  17541.000000  6.950000
stuart berman                 jazz   11316.000000  7.700000
                              metal  15086.500000  6.378571
taylor m. clark               jazz    7759.000000  7.250000
                              metal   1700.000000  8.000000
tim finney                    jazz   15403.666667  7.300000
tom breihan                   jazz   10696.000000  7.050000
                              metal  12961.291667  6.679167
william bowers                metal   5790.600000  5.800000
zach baron                    jazz    4032.000000  6.700000
                              metal   8979.826087  7.065217
zach hooker                   jazz    5403.750000  7.675000
zach kelly                    metal  15969.000000  5.666667
zoe camp                      jazz   22015.000000  7.600000
                              metal  20596.086957  7.217391

[240 rows x 2 columns]

In [114]:
df_genre_reviews.groupby(['genre', 'author']).mean()

# data.groupby(func, axis=0).mean()
# data.groupby(['col1', 'col2'])['col3'].mean()

reviewid      score
genre author                                           
jazz  aaron leitko              16213.600000   7.600000
      adam moerder               5427.000000   2.700000
      al shipley                 7003.000000   5.450000
      alex linhardt              2017.500000   8.150000
      alexander lloyd linhardt   4770.000000   8.066667
      alison fields              5395.000000   6.500000
      amanda petrusich          21797.000000   9.000000
      andrew bryant              3348.500000   6.750000
      andrew gaerig             13740.666667   5.966667
      andy battaglia            12361.500000   6.700000
      andy beta                  8147.200000   7.780000
      andy o'connor             18459.000000   7.700000
      ben westhoff              11114.000000   7.500000
      brad haywood               3413.500000   4.000000
      brad nelson               21105.500000   7.250000
      brandon stosuy             7545.600000   7.540000
      brendan reid               1510.000000   8.000000
      brent s. sirota            5216.500000   8.250000
      brian howe                11972.750000   6.915000
      cameron macdonald          1437.500000   7.250000
      camilo arturo leslie       4385.000000   7.900000
      carvell wallace           22159.000000  10.000000
      chris dahlen               5573.869565   7.808696
      christopher dare           7329.400000   6.920000
      cory d. byrom              2388.000000   6.000000
      cosmo lee                 12689.666667   6.666667
      david m. pecoraro          5371.200000   7.600000
      david moore                5747.000000   7.700000
      david raposa              10997.000000   5.100000
      dean van nguyen           22639.000000   8.700000
...                                      ...        ...
metal peter macia                1693.500000   7.900000
      rebecca raber             13588.000000   7.100000
      rob mitchum                2518.166667   5.900000
      robbie mackey             11580.666667   6.866667
      rollie pemberton           3731.500000   3.550000
      roque strew               11759.000000   7.900000
      ryan dombal                2780.000000   3.700000
      ryan kearney               4242.000000   5.900000
      ryan schreiber             3822.250000   6.850000
      saby reyes kulkarni       22502.000000   6.800000
      saby reyes-kulkarni       21760.684211   6.973684
      sam chennault              5102.500000   7.350000
      sam hockley-smith         14743.000000   6.800000
      sam lefebvre              20383.000000   6.500000
      sam sodomsky              22259.500000   7.525000
      sam ubl                    4427.363636   6.472727
      savy reyes-kulkarni       22559.000000   7.700000
      scott hreha                8099.000000   8.400000
      scott plagenhoef           8616.000000   8.200000
      sean fennessey            14515.000000   8.600000
      spencer owen               2860.000000   8.500000
      stephen m. deusner        13522.000000   6.450000
      steven hyden              17541.000000   6.950000
      stuart berman             15086.500000   6.378571
      taylor m. clark            1700.000000   8.000000
      tom breihan               12961.291667   6.679167
      william bowers             5790.600000   5.800000
      zach baron                 8979.826087   7.065217
      zach kelly                15969.000000   5.666667
      zoe camp                  20596.086957   7.217391

[240 rows x 2 columns]

In [181]:
df_genre_reviews.head(10)

,reviewid,score,author,genre
0,1509,7.9,brent s. sirota,jazz
1,9421,6.0,mark richardson,jazz
2,3833,7.6,mark richardson,jazz
3,7120,7.2,matthew murphy,jazz
4,6885,4.0,jason crock,metal
5,11895,7.1,marc masters,metal
6,8383,9.7,alex linhardt,metal
7,6186,6.5,andrew bryant,metal
8,14716,7.8,nate patrin,jazz
9,15978,7.7,brandon stosuy,metal


In [ ]:
df.genre_reviews.[(df['column_name'] >= A) & (df['column_name'] <= B)]

In [134]:
#execute simple commands
pysqldf = lambda q: sqldf(q, globals())

In [146]:
q = """SELECT author FROM df_genre_reviews WHERE genre =['metal'];"""

In [147]:
results = pysqldf(q)
results.describe()

PandaSQLException: (sqlite3.OperationalError) no such table: 'metal', 'jazz' [SQL: "SELECT author FROM df_genre_reviews WHERE genre IN ['metal', 'jazz'];"] (Background on this error at: http://sqlalche.me/e/e3q8)

In [84]:
x

In [ ]:
#execute simple commands
reviews = cur.execute("""
SELECT * FROM reviews;
""")

In [ ]:
df_genre_reviews.columns = ['reviewid', 'score', 'genre']

In [ ]:
df_genre_reviews.groupby(['genre']).mean()

In [161]:
df_jazz = df_genre_reviews[df_genre_reviews['genre'] =='jazz']

In [163]:
df_jazz.head(5)

,reviewid,score,author,genre
0,1509,7.9,brent s. sirota,jazz
1,9421,6.0,mark richardson,jazz
2,3833,7.6,mark richardson,jazz
3,7120,7.2,matthew murphy,jazz
8,14716,7.8,nate patrin,jazz


In [164]:
df_metal = df_genre_reviews[df_genre_reviews['genre'] =='metal']

In [166]:
df_metal.head(5)

,reviewid,score,author,genre
4,6885,4.0,jason crock,metal
5,11895,7.1,marc masters,metal
6,8383,9.7,alex linhardt,metal
7,6186,6.5,andrew bryant,metal
9,15978,7.7,brandon stosuy,metal


In [171]:
df_merged = pd.merge(df_metal, df_jazz, how='inner', on='author')

#### TEST

In [14]:
#execute simple commands
genres = cur.execute("""
SELECT reviews.reviewid, reviews.score, reviews.author, genres.genre FROM reviews
INNER JOIN genres ON reviews.reviewid = genres.reviewid
WHERE genres.genre = 'metal' OR genres.genre = 'jazz' GROUP BY reviews.reviewid, reviews.score, reviews.author, genres.genre;
""")

UndefinedTable: missing FROM-clause entry for table "review"
LINE 4: ...genre = 'metal' OR genres.genre = 'jazz' GROUP BY review.aut...
                                                             ^


In [ ]:
#execute simple commands
query = cur.execute("""
SELECT AVG(score), COUNT(score) FROM reviews GROUP BY author;
""")


In [ ]:
#execute simple commands
data = cur.execute("""
SELECT reviews.reviewid, reviews.score, genres.genre FROM reviews
INNER JOIN genres ON reviews.reviewid = genres.reviewid;
""")

In [ ]:
data = cur.fetchall()
#load cast genres into a dataframe
df_data = pd.DataFrame(data)

In [ ]:
df_data.describe(), df_data.info()

In [ ]:
df_data.head()

In [ ]:
df_data.columns = ['reviewid', 'score', 'genre']

In [ ]:
df_data.head()

In [ ]:
# summary statistics for score (dep var)
df_data.score.describe()

In [ ]:
df_data.head(10)

In [ ]:
score = df_data['score']

In [ ]:
log_score = np.log(df_data['score'])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8))
_ = ax[0].hist(score, bins=50)
# _ = ax[1].hist(log_score, bins=50)

In [ ]:
df_genre_reviews.describe()

In [ ]:
# query with an f-string
query = f"""
SELECT {', '.join(columns)} FROM dogs;
"""

cur.execute(query)

In [ ]:
# option 1 to display, using fetchall method
data = cur.fetchall()
data

In [ ]:
# option 1 to display, using fetchall method
temp_data = cur.fetchall()
temp_data

In [ ]:
# query with an f-string
query = f"""
SELECT * FROM genres;
"""

cur.execute(query)


In [ ]:
temp_data = cur.fetchall()
temp_data

#### Exploring years table

In [247]:
#execute simple commands
cur.execute("""
SELECT * FROM labels;
""")

In [248]:
j = cur.fetchall()
#load cast genres into a dataframe
df = pd.DataFrame(j)

In [255]:
df.columns = [i[0] for i in cur.description]
df.head()

,reviewid,label
0,22703,virgin
1,22721,hathenter
2,22659,static shock
3,22659,fashionable idiots
4,22661,kill rock stars


In [250]:
len(df)

20190

In [258]:
df.label.unique

<bound method Series.unique of 0        virgin                
1        hathenter             
2        static shock          
3        fashionable idiots    
4        kill rock stars       
5        mississippi           
6        teklife               
7        rca                   
8        age 101               
9        stroboscopic artefacts
10       warp                  
11       roc nation            
12       important             
13       self-released         
14       pan                   
15       awge                  
16       a country called earth
17       babygrande            
18       soul jazz             
19       cejero                
20       self-released         
21       leaving               
22       radio killa           
23       profound lore         
24       self-released         
25       northern spy          
26       p.w. elverum & sun    
27       bedroom community     
28       mercury               
29       polyvinyl             
         

In [257]:
df['label'].value_counts

<bound method IndexOpsMixin.value_counts of 0        virgin                
1        hathenter             
2        static shock          
3        fashionable idiots    
4        kill rock stars       
5        mississippi           
6        teklife               
7        rca                   
8        age 101               
9        stroboscopic artefacts
10       warp                  
11       roc nation            
12       important             
13       self-released         
14       pan                   
15       awge                  
16       a country called earth
17       babygrande            
18       soul jazz             
19       cejero                
20       self-released         
21       leaving               
22       radio killa           
23       profound lore         
24       self-released         
25       northern spy          
26       p.w. elverum & sun    
27       bedroom community     
28       mercury               
29       polyvinyl          